In [8]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['MCA_WT']

# Define and insert data for the 'projects' collection
projects = db['projects']
projects.delete_many({})
projects_data = [
    {"_id": 1, "project_name": "AI Development", "project_type": "Technology", "duration_months": 6, "employees": [1, 3, 5]},
    {"_id": 2, "project_name": "Market Research", "project_type": "Marketing", "duration_months": 4, "employees": [2,5]},
    {"_id": 3, "project_name": "Website Overhaul", "project_type": "Technology", "duration_months": 2, "employees": [1]},
    {"_id": 4, "project_name": "Product Launch", "project_type": "Sales", "duration_months": 5, "employees": [2,4]},
    {"_id": 5, "project_name": "Customer Survey", "project_type": "Marketing", "duration_months": 3, "employees": [3,5]}
]
projects.insert_many(projects_data)

# Define and insert data for the 'employees' collection
employees = db['employees']
employees.delete_many({})
employees_data = [
    {"_id": 1, "name": "Mr. Patil", "projects": [1, 3]},
    {"_id": 2, "name": "Ms. Sharma", "projects": [2, 4]},
    {"_id": 3, "name": "Mr. Gupta", "projects": [1, 5]},
    {"_id": 4, "name": "Ms. Desai", "projects": [4]},
    {"_id": 5, "name": "Mr. Rao", "projects": [1, 2, 5]}
]
employees.insert_many(employees_data)




InsertManyResult([1, 2, 3, 4, 5], acknowledged=True)

In [3]:
# List all names of projects where project_type = ...
def list_projects_by_type(project_type):
    result = projects.find({"project_type": project_type}, {"project_name": 1, "_id": 0})
    return [project["project_name"] for project in result]

# Example usage:
print("Projects with type 'Marketing':", list_projects_by_type("Marketing"))


Projects with type 'Marketing': ['Market Research', 'Customer Survey']


In [4]:
# List all the projects with duration greater than 3 months
def list_projects_duration_gt(months):
    result = projects.find({"duration_months": {"$gt": months}}, {"project_name": 1, "_id": 0})
    return [project["project_name"] for project in result]

# Example usage:
print("Projects with duration > 3 months:", list_projects_duration_gt(3))


Projects with duration > 3 months: ['AI Development', 'Market Research', 'Product Launch']


In [11]:
# Count the number of employees working on a specific project
def count_employees_on_project(name):
    emps = projects.find_one({"project_name":name},{'employees':1, '_id':0})
    emp_ids = emps['employees']
    emp_names = employees.find({'_id':{'$in':emp_ids}}, {"name":1,"_id":0})

    return [ name['name'] for name in emp_names]
# Example usage:
project = 'AI Development'
print("Number of employees working on project 1 (AI Development):", count_employees_on_project(project))


Number of employees working on project 1 (AI Development): ['Mr. Patil', 'Mr. Gupta', 'Mr. Rao']


In [6]:
# . List the names of projects on which "Mr. Patil" is working
def list_projects_for_employee(employee_name):
    employee = employees.find_one({"name": employee_name})
    if employee:
        project_ids = employee.get("projects", [])
        result = projects.find({"_id": {"$in": project_ids}}, {"project_name": 1, "_id": 0})
        return [project["project_name"] for project in result]
    return []

# Example usage:
print("Projects Mr. Patil is working on:", list_projects_for_employee("Mr. Patil"))


Projects Mr. Patil is working on: ['AI Development', 'Website Overhaul']
